# Orthogonal Matching Pursuit (OMP)

OMP is a greedy algorithm for sparse signal recovery. It iteratively selects the most correlated component of the measurement vector with the residual and updates the support set.

## **Mathematical Steps**

1. **Input**:
   - Measurement matrix $ \mathbf{A} \in \mathbb{R}^{m \times n} $.
   - Measurement vector $ \mathbf{y} \in \mathbb{R}^{m} $.
   - Sparsity level $ k $.

2. **Initialization**:
   - Residual $ \mathbf{r}_0 = \mathbf{y} $.
   - Support set $ \mathcal{S}_0 = \emptyset $.
   - Iteration counter $ t = 1 $.

3. **Iteration**:
   - **Step 1: Find the index of the maximum correlation**:
     $
     i_t = \arg\max_{j} |\mathbf{A}_j^T \mathbf{r}_{t-1}|,
     $
     where $ \mathbf{A}_j $ is the $ j $-th column of $ \mathbf{A} $.

   - **Step 2: Update the support set**:
     $
     \mathcal{S}_t = \mathcal{S}_{t-1} \cup \{i_t\}.
     $

   - **Step 3: Estimate the signal**:
     Solve the least-squares problem:
     $
     \mathbf{x}_t = \arg\min_{\mathbf{x}} \|\mathbf{y} - \mathbf{A}_{\mathcal{S}_t} \mathbf{x}\|_2,
     $
     where $ \mathbf{A}_{\mathcal{S}_t} $ is the submatrix of $ \mathbf{A} $ with columns indexed by $ \mathcal{S}_t $.

   - **Step 4: Update the residual**:
     $
     \mathbf{r}_t = \mathbf{y} - \mathbf{A}_{\mathcal{S}_t} \mathbf{x}_t.
     $

   - **Step 5: Check stopping criterion**:
     If $ t = k $ or $ \|\mathbf{r}_t\|_2 $ is small, stop. Otherwise, increment $ t $ and repeat.

4. **Output**:
   - Sparse signal estimate $ \mathbf{x} $ with non-zero elements at indices $ \mathcal{S}_k $.

---

# Subspace Pursuit (SP)

Subspace Pursuit is an iterative algorithm that improves upon OMP by considering multiple indices in each iteration and refining the support set.

## **Mathematical Steps**

1. **Input**:
   - Measurement matrix $ \mathbf{A} \in \mathbb{R}^{m \times n} $.
   - Measurement vector $ \mathbf{y} \in \mathbb{R}^{m} $.
   - Sparsity level $ k $.

2. **Initialization**:
   - Compute the correlation vector $ \mathbf{c} = \mathbf{A}^T \mathbf{y} $.
   - Select the $ k $ indices with the largest magnitudes in $ \mathbf{c} $ to form the initial support set $ \mathcal{S}_0 $.

3. **Iteration**:
   - **Step 1: Estimate the signal**:
     Solve the least-squares problem:
     $
     \mathbf{x}_t = \arg\min_{\mathbf{x}} \|\mathbf{y} - \mathbf{A}_{\mathcal{S}_{t-1}} \mathbf{x}\|_2.
     $

   - **Step 2: Prune the support set**:
     Retain the $ k $ indices corresponding to the largest magnitudes in $ \mathbf{x}_t $, forming a new support set $ \mathcal{S}_t'$ .

   - **Step 3: Expand the support set**:
     Compute the residual $ \mathbf{r}_t = \mathbf{y} - \mathbf{A}_{\mathcal{S}_t'} \mathbf{x}_t $.
     Find the $ k $ indices with the largest magnitudes in $ \mathbf{A}^T \mathbf{r}_t $, and add them to $ \mathcal{S}_t'$ to form $ \mathcal{S}_t $.

   - **Step 4: Check stopping criterion**:
     If $ \|\mathbf{r}_t\|_2 $ is small or the support set stabilizes, stop. Otherwise, repeat.

4. **Output**:
   - Sparse signal estimate $ \mathbf{x} $ with non-zero elements at indices $ \mathcal{S}_t $.

---

# Sequential Pursuit

Sequential Pursuit is a greedy algorithm that sequentially selects indices based on correlation and updates the residual.

## **Mathematical Steps**

1. **Input**:
   - Measurement matrix $ \mathbf{A} \in \mathbb{R}^{m \times n} $.
   - Measurement vector $ \mathbf{y} \in \mathbb{R}^{m} $.
   - Sparsity level $ k $.

2. **Initialization**:
   - Residual $ \mathbf{r}_0 = \mathbf{y} $.
   - Support set $ \mathcal{S}_0 = \emptyset $.
   - Iteration counter $ t = 1 $.

3. **Iteration**:
   - **Step 1: Find the index of the maximum correlation**:
     $
     i_t = \arg\max_{j} |\mathbf{A}_j^T \mathbf{r}_{t-1}|.
     $

   - **Step 2: Update the support set**:
     $
     \mathcal{S}_t = \mathcal{S}_{t-1} \cup \{i_t\}.
     $

   - **Step 3: Estimate the signal**:
     Solve the least-squares problem:
     $
     \mathbf{x}_t = \arg\min_{\mathbf{x}} \|\mathbf{y} - \mathbf{A}_{\mathcal{S}_t} \mathbf{x}\|_2.
     $

   - **Step 4: Update the residual**:
     $
     \mathbf{r}_t = \mathbf{y} - \mathbf{A}_{\mathcal{S}_t} \mathbf{x}_t.
     $

   - **Step 5: Check stopping criterion**:
     If $ t = k $ or $ \|\mathbf{r}_t\|_2 $ is small, stop. Otherwise, increment $ t $ and repeat.

4. **Output**:
   - Sparse signal estimate $ \mathbf{x} $ with non-zero elements at indices $ \mathcal{S}_k $.

---

## **Key Differences**
- **OMP**: Adds one index per iteration and solves a least-squares problem to update the signal estimate.
- **SP**: Considers multiple indices per iteration and refines the support set by pruning and expanding.
- **Sequential Pursuit**: Similar to OMP but may differ in the stopping criterion or update rules.